In [1]:
import os
import tweepy as tw
import pandas as pd
from textblob import TextBlob
import json

In [2]:
import tweepy
from tweepy.error import TweepError
from tweepy.models import Status

In [3]:
tweets = []

# Open connection to file
with open("Impeach_tweets_noon.txt", "r") as tweets_file:
    # Read in tweets and store in list
    for line in tweets_file:
        tweet = json.loads(line)
        tweets.append(tweet)

In [4]:
tweet

{'created_at': 'Tue Feb 09 17:36:53 +0000 2021',
 'id': 1359194573373902857,
 'id_str': '1359194573373902857',
 'text': 'https://t.co/usj6xbBti0',
 'display_text_range': [0, 0],
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 6972272,
  'id_str': '6972272',
  'name': 'Brandon Rohwer',
  'screen_name': 'brandonRohwer',
  'location': 'FROM PARTS UNKNOWN (Brooklyn)',
  'url': 'http://www.brandonrohwer.com',
  'description': "you can't spell #onbrand without brandon",
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 1364,
  'friends_count': 774,
  'listed_count': 44,
  'favourites_count': 25216,
  'statuses_count': 52428,
  'created_at': 'Wed Jun 20 17:05:27 +0000 2007',
  'utc_offset': None,
  'time_zone': No

In [5]:
df = pd.DataFrame(tweets, columns=['created_at', 'text', 'source', 'possibly_sensitive', 'lang'])
df.head()

,created_at,text,source,possibly_sensitive,lang
0,Tue Feb 09 17:01:28 +0000 2021,RT @JesseBWatters: If you put Maxine Waters wo...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,en
1,Tue Feb 09 17:01:28 +0000 2021,Lindsey Graham Threatens to Call Democrats as ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",False,en
2,Tue Feb 09 17:01:28 +0000 2021,@NunesAlt As long as it's not Trump streaking....,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,en
3,Tue Feb 09 17:01:28 +0000 2021,RT @chipfranklin: Who else thinks the vote to ...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,en
4,Tue Feb 09 17:01:28 +0000 2021,RT @JaySekulow: HAPPENING NOW: Impeachment Tri...,"<a href=""http://twitter.com/download/iphone"" r...",False,en


In [6]:
df['created_at'] = pd.to_datetime(df.created_at)

In [7]:
df['source'] = df['source'].str.extract('>(.+?)<', expand=False).str.strip() 

In [8]:
df = df.rename(columns = {'created_at': 'Time Created', 'text': 'Tweet Text', 'source': 'Source'
                    , 'possibly_sensitive': 'Possibly Sensitive', 'lang': 'Language'})

In [9]:
df.to_csv('Impeach_tweets.csv', index = False)

In [10]:
df['Possibly Sensitive'].unique()

array([nan, False, True], dtype=object)

In [15]:
cleaned_df = df.copy()

In [16]:
cleaned_df

,Time Created,Tweet Text,Source,Possibly Sensitive,Language
0,2021-02-09 17:01:28+00:00,RT @JesseBWatters: If you put Maxine Waters wo...,Twitter for iPhone,NaN,en
1,2021-02-09 17:01:28+00:00,Lindsey Graham Threatens to Call Democrats as ...,Twitter Web App,False,en
2,2021-02-09 17:01:28+00:00,@NunesAlt As long as it's not Trump streaking....,Twitter Web App,NaN,en
3,2021-02-09 17:01:28+00:00,RT @chipfranklin: Who else thinks the vote to ...,Twitter Web App,NaN,en
4,2021-02-09 17:01:28+00:00,RT @JaySekulow: HAPPENING NOW: Impeachment Tri...,Twitter for iPhone,False,en
...,...,...,...,...,...
99996,2021-02-09 17:36:53+00:00,"RT @FranklinH3000: It took Martin van Buren, h...",TweetDeck,NaN,en
99997,2021-02-09 17:36:53+00:00,RT @jason_meister: This #impeachment trial is ...,Twitter for iPhone,NaN,en
99998,2021-02-09 17:36:53+00:00,RT @chipfranklin: Who else thinks the vote to ...,Twitter for iPhone,NaN,en
99999,2021-02-09 17:36:53+00:00,The first day of the Senate impeachment trial ...,SocialFlow,False,en


In [17]:
import re

In [18]:
for i in range(len(cleaned_df['Tweet Text'])):
    cleaned_df['Tweet Text'][i] = " ".join([word for word in cleaned_df['Tweet Text'][i].split()
                              if 'http' not in word and '@' not in word and '<' not in word])

# remove special characters
cleaned_df['Tweet Text'] = cleaned_df['Tweet Text'].apply(lambda x: re.sub('[!@#$:).;,?&-]', '', x))
cleaned_df['Tweet Text'] = cleaned_df['Tweet Text'].apply(lambda x: re.sub('  ', ' ', x))

<ipython-input-18-792820305062>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['Tweet Text'][i] = " ".join([word for word in cleaned_df['Tweet Text'][i].split()


In [19]:
cleaned_df = cleaned_df[cleaned_df['Possibly Sensitive'] != True]

In [20]:
cleaned_df

,Time Created,Tweet Text,Source,Possibly Sensitive,Language
0,2021-02-09 17:01:28+00:00,RT If you put Maxine Waters words in Trump's m...,Twitter for iPhone,NaN,en
1,2021-02-09 17:01:28+00:00,Lindsey Graham Threatens to Call Democrats as ...,Twitter Web App,False,en
2,2021-02-09 17:01:28+00:00,As long as it's not Trump streaking I don't th...,Twitter Web App,NaN,en
3,2021-02-09 17:01:28+00:00,RT Who else thinks the vote to convict Trump i...,Twitter Web App,NaN,en
4,2021-02-09 17:01:28+00:00,RT HAPPENING NOW Impeachment Trial Begins,Twitter for iPhone,False,en
...,...,...,...,...,...
99996,2021-02-09 17:36:53+00:00,RT It took Martin van Buren himself not a stri...,TweetDeck,NaN,en
99997,2021-02-09 17:36:53+00:00,RT This impeachment trial is proof that we liv...,Twitter for iPhone,NaN,en
99998,2021-02-09 17:36:53+00:00,RT Who else thinks the vote to convict Trump i...,Twitter for iPhone,NaN,en
99999,2021-02-09 17:36:53+00:00,The first day of the Senate impeachment trial ...,SocialFlow,False,en


In [21]:
cleaned_df.to_csv('Impeach_tweets_clean.csv', index = False)

In [22]:
sentiment_df = cleaned_df.copy()

In [23]:
sentiment_df[['polarity', 'subjectivity']] = sentiment_df['Tweet Text'].apply(lambda text: pd.Series(TextBlob(text).sentiment))

In [24]:
conditions = [(sentiment_df['polarity'] > 0),
              (sentiment_df['polarity'] == 0),
              (sentiment_df['polarity'] < 0)]

In [25]:
values = ['positive', 'neutral', 'negative']

In [27]:
import numpy as np

In [28]:
sentiment_df['sentiment'] = np.select(conditions, values)

In [29]:
sentiment_df.head()

,Time Created,Tweet Text,Source,Possibly Sensitive,Language,polarity,subjectivity,sentiment
0,2021-02-09 17:01:28+00:00,RT If you put Maxine Waters words in Trump's m...,Twitter for iPhone,NaN,en,0.00,0.0,neutral
1,2021-02-09 17:01:28+00:00,Lindsey Graham Threatens to Call Democrats as ...,Twitter Web App,False,en,0.00,0.0,neutral
2,2021-02-09 17:01:28+00:00,As long as it's not Trump streaking I don't th...,Twitter Web App,NaN,en,-0.05,0.4,negative
3,2021-02-09 17:01:28+00:00,RT Who else thinks the vote to convict Trump i...,Twitter Web App,NaN,en,-0.40,0.7,negative
4,2021-02-09 17:01:28+00:00,RT HAPPENING NOW Impeachment Trial Begins,Twitter for iPhone,False,en,0.00,0.0,neutral


In [30]:
sentiment_df.to_csv('Impeach_tweets_sentiment.csv', index = False)